# Final Project

## 파이썬 환경 설정 및 확인

In [ ]:
# C:/Users/TECH2_22/이도원/usingcloud_final/venv/Scripts/python.exe

In [20]:
import os

os.getcwd()

'c:\\Users\\TECH2_22\\이도원\\usingcloud_final'

In [1]:
print('hello world')

hello world


In [2]:
!python --version

Python 3.8.8


## 데이터 로드 및 설정

### 최초 1회만 실행

In [1]:
import pandas as pd
import numpy as np
import json

with open('./data/train_original.json', encoding='utf-8') as train_f:
    train_data = json.loads(train_f.read())
    train_df = pd.DataFrame(train_data['documents'])

with open('./data/valid_original.json', encoding='utf-8') as valid_f:
    valid_data = json.loads(valid_f.read())
    valid_df = pd.DataFrame(valid_data['documents'])

## eda

In [69]:
train_df.head(3)

,id,category,media_type,media_sub_type,media_name,size,char_count,publish_date,title,text,annotator_id,document_quality_scores,extractive,abstractive
0,290741778,종합,online,지역지,광양신문,small,927,2018-01-05 18:54:55,"논 타작물 재배, 2월 말까지 신청하세요","[[{'index': 0, 'sentence': 'ha당 조사료 400만원…작물별 ...",11,"{'readable': 4, 'accurate': 3, 'informative': ...","[2, 3, 10]",[전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 ...
1,290741792,종합,online,지역지,광양신문,small,764,2018-01-05 18:53:20,"여수시, 컬러빌리지 마무리...‘색채와 빛’도시 완성","[[{'index': 0, 'sentence': '8억 투입, 고소천사벽화·자산마을...",12,"{'readable': 4, 'accurate': 4, 'informative': ...","[2, 4, 11]",[여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천...
2,290741793,스포츠,online,지역지,광양신문,medium,1066,2018-01-05 18:52:15,“새해 정기 받고 올해는 반드시 일내자!”,"[[{'index': 0, 'sentence': '전남드래곤즈 해맞이 다짐…선수 영...",13,"{'readable': 4, 'accurate': 4, 'informative': ...","[3, 5, 7]",[전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구...


In [59]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243983 entries, 0 to 243982
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   id                       243983 non-null  object
 1   category                 243983 non-null  object
 2   media_type               243983 non-null  object
 3   media_sub_type           243983 non-null  object
 4   media_name               243983 non-null  object
 5   size                     243983 non-null  object
 6   char_count               243983 non-null  object
 7   publish_date             243983 non-null  object
 8   title                    243983 non-null  object
 9   text                     243983 non-null  object
 10  annotator_id             243983 non-null  int64 
 11  document_quality_scores  243983 non-null  object
 12  extractive               243983 non-null  object
 13  abstractive              243983 non-null  object
dtypes: int64(1), object(

In [61]:
train_df.shape

(243983, 14)

In [70]:
train_df.head(1).T

,0
id,290741778
category,종합
media_type,online
media_sub_type,지역지
media_name,광양신문
size,small
char_count,927
publish_date,2018-01-05 18:54:55
title,"논 타작물 재배, 2월 말까지 신청하세요"
text,"[[{'index': 0, 'sentence': 'ha당 조사료 400만원…작물별 ..."


In [62]:
train_df.isnull().sum()

id                         0
category                   0
media_type                 0
media_sub_type             0
media_name                 0
size                       0
char_count                 0
publish_date               0
title                      0
text                       0
annotator_id               0
document_quality_scores    0
extractive                 0
abstractive                0
dtype: int64

In [63]:
train_df.columns

Index(['id', 'category', 'media_type', 'media_sub_type', 'media_name', 'size',
       'char_count', 'publish_date', 'title', 'text', 'annotator_id',
       'document_quality_scores', 'extractive', 'abstractive'],
      dtype='object')

## 전처리

In [44]:
def sentence_validation(item, sentence, item_list):
    val_bool = False
    if item == []:
        val_bool = True
    if sentence in item_list:
        val_bool = True
    if len(sentence) <= 20:
        val_bool = True
    if '@' in sentence or '/사진' in sentence:
        val_bool = True
    return val_bool


def extract_body(article) -> str:
    item_list = []

    # item = train_df['text'][n][idx]
    for idx, item in enumerate(article):
        sentence = item[0]['sentence']
        if sentence_validation(item, sentence, item_list):
            print(f'넘기는 항목 : {sentence}')
            continue
        
        if len(item) >= 2:
            for sub_item in item:
                
                print()
                sub_sentence = sub_item['sentence']
                print(f'검증대상 하위 항목 : {sub_sentence}')
                if not sentence_validation(sub_item, sentence, item_list):
                    print(f'추가되는 하위 항목 : {sentence}')
                    item_list.append(sub_item['sentence'])
                    continue
                else:
                    print(f'넘기는 하위 항목: {sub_sentence}')
        
        print(f'추가되는 항목 : {sentence}')
        item_list.append(sentence)

    return ' '.join(item_list)

In [45]:
sentence_list = extract_body(train_df['text'][24000])
sentence_list

넘기는 항목 : 박종대 기자
넘기는 항목 : 도내 거리 악취 없애려 안간힘
추가되는 항목 : 경기도내 지자체들이 매년 반복되는 가을철 은행나무 악취 민원 해결을 위해 온갖 방법을 동원하면서 시민들의 불편을 줄이고자 안간힘을 쓰고 있다.
추가되는 항목 : 6일 도내 지자체들에 따르면 친환경적인 방법으로 은행열매를 제거해 가을철 도로변 악취 문제를 해결하기 위한 방법을 고안하고 있다.
추가되는 항목 : 대표적으로 수원시는 산하 사업소를 통해 주요 대로변, 상가밀집지역 등 유동인구가 많은 곳을 대상으로 ‘은행 암나무 수종(樹種) 교체사업’을 진행한다.

검증대상 하위 항목 : 시는 지난해 은행나무를 전수조사하고 수종 교체를 위한 연차별 기본계획을 수립한 바 있다.
추가되는 하위 항목 : 시는 지난해 은행나무를 전수조사하고 수종 교체를 위한 연차별 기본계획을 수립한 바 있다.

검증대상 하위 항목 : 지난 4월 사업을 시작해 은행 암나무 600여 그루를 은행 수나무·느티나무 등 다른 나무로 교체했다.
넘기는 하위 항목: 지난 4월 사업을 시작해 은행 암나무 600여 그루를 은행 수나무·느티나무 등 다른 나무로 교체했다.

검증대상 하위 항목 : 2022년까지 예산 36억 원을 투입할 예정이다.
넘기는 하위 항목: 2022년까지 예산 36억 원을 투입할 예정이다.

검증대상 하위 항목 : 아직 교체하지 못 했거나 보존 가치가 있는 은행나무는 열매 털기 작업을 실시한다.
넘기는 하위 항목: 아직 교체하지 못 했거나 보존 가치가 있는 은행나무는 열매 털기 작업을 실시한다.

검증대상 하위 항목 : 은행열매 진동 수확기를 활용해 은행열매를 털어 악취를 없앤다.
넘기는 하위 항목: 은행열매 진동 수확기를 활용해 은행열매를 털어 악취를 없앤다.
추가되는 항목 : 시는 지난해 은행나무를 전수조사하고 수종 교체를 위한 연차별 기본계획을 수립한 바 있다.

검증대상 하위 항목 : 은행나무 열매와 낙엽을 한꺼번에 처리할 수 있는 ‘은행열매 수집망’도 설치했다.
추가되는 하위 항목 : 

'경기도내 지자체들이 매년 반복되는 가을철 은행나무 악취 민원 해결을 위해 온갖 방법을 동원하면서 시민들의 불편을 줄이고자 안간힘을 쓰고 있다. 6일 도내 지자체들에 따르면 친환경적인 방법으로 은행열매를 제거해 가을철 도로변 악취 문제를 해결하기 위한 방법을 고안하고 있다. 대표적으로 수원시는 산하 사업소를 통해 주요 대로변, 상가밀집지역 등 유동인구가 많은 곳을 대상으로 ‘은행 암나무 수종(樹種) 교체사업’을 진행한다. 시는 지난해 은행나무를 전수조사하고 수종 교체를 위한 연차별 기본계획을 수립한 바 있다. 시는 지난해 은행나무를 전수조사하고 수종 교체를 위한 연차별 기본계획을 수립한 바 있다. 은행나무 열매와 낙엽을 한꺼번에 처리할 수 있는 ‘은행열매 수집망’도 설치했다. 은행나무 열매와 낙엽을 한꺼번에 처리할 수 있는 ‘은행열매 수집망’도 설치했다. 안산시 단원구는 지난달 21일부터 은행나무 열매로 인한 지속적인 악취로 고통받는 시민 피해를 막기 위해 은행열매를 조기 채취하고 있다. 안산시 단원구는 지난달 21일부터 은행나무 열매로 인한 지속적인 악취로 고통받는 시민 피해를 막기 위해 은행열매를 조기 채취하고 있다. 구는 수거된 은행열매에 대해 중금속검사와 잔류농약검사를 벌여 ‘안전하다’고 판정된 수확물은 지역 복지관 및 경로당에 기부할 예정이다. 구는 수거된 은행열매에 대해 중금속검사와 잔류농약검사를 벌여 ‘안전하다’고 판정된 수확물은 지역 복지관 및 경로당에 기부할 예정이다. 부천시와 고양시 덕양구 역시 은행나무 암나무를 열매를 맺지 않는 수나무로 바꾼다. 부천시와 고양시 덕양구 역시 은행나무 암나무를 열매를 맺지 않는 수나무로 바꾼다. 최광열 수원시 공원녹지사업소장은 "가을철에 접어들면서 버스정류장, 상가밀집지역 등을 중심으로 은행열매 악취를 호소하는 민원이 폭주하고 있다"며 "시민 불편을 줄이기 위해 총력을 기울이겠다"고 말했다.'

In [10]:
for item in train_df['text'][24000]:
    print(item)

[{'index': 0, 'sentence': '박종대 기자', 'highlight_indices': ''}]
[{'index': 1, 'sentence': '도내 거리 악취 없애려 안간힘', 'highlight_indices': ''}, {'index': 2, 'sentence': '은행 열매 수집망.', 'highlight_indices': ''}, {'index': 3, 'sentence': '/사진 = 수원시 제공', 'highlight_indices': ''}]
[{'index': 4, 'sentence': '경기도내 지자체들이 매년 반복되는 가을철 은행나무 악취 민원 해결을 위해 온갖 방법을 동원하면서 시민들의 불편을 줄이고자 안간힘을 쓰고 있다.', 'highlight_indices': '11,13;41,43'}]
[{'index': 5, 'sentence': '6일 도내 지자체들에 따르면 친환경적인 방법으로 은행열매를 제거해 가을철 도로변 악취 문제를 해결하기 위한 방법을 고안하고 있다.', 'highlight_indices': ''}]
[{'index': 6, 'sentence': '대표적으로 수원시는 산하 사업소를 통해 주요 대로변, 상가밀집지역 등 유동인구가 많은 곳을 대상으로 ‘은행 암나무 수종(樹種) 교체사업’을 진행한다.', 'highlight_indices': ''}]
[{'index': 7, 'sentence': '시는 지난해 은행나무를 전수조사하고 수종 교체를 위한 연차별 기본계획을 수립한 바 있다.', 'highlight_indices': ''}, {'index': 8, 'sentence': '지난 4월 사업을 시작해 은행 암나무 600여 그루를 은행 수나무·느티나무 등 다른 나무로 교체했다.', 'highlight_indices': '44,46'}, {'index': 9, 'sentence': '2022년까지 예산 36억 원을 투입할 예정이다.', 'highlight_indices': ''}, {'index': 10, 'sen

## 여기가 끝